In [1]:
import math
import numpy as np
import mysql.connector

In [2]:
def db_connect(query):
    mydb = mysql.connector.connect(
        host='localhost',
        user='root',
        password='123456',
        # scale = 0.025
        database='tpc-h-0.05',
        auth_plugin='mysql_native_password'
    )
    cursor = mydb.cursor()
    cursor.execute(query)
    return cursor.fetchall()

In [3]:
def get_query_result():
    
    # Q18  

    sql = (
        "select c_custkey, count(*) as count "
        "from customer, orders, lineitem "
        "where c_custkey = o_custkey "
        "and l_orderkey = o_orderkey "
        "group by c_custkey "
        "order by count "
    )

    '''
    # Q12
    sql = (
        "select o_orderkey, count(*) as count "
        "from orders, lineitem "
        "where o_orderkey = l_orderkey "
        "group by o_orderkey "
        "order by count"
    )
    
    '''
    '''
    # Q9
    sql = (
        "select s_suppkey, count(*) as count "
        "from supplier, partsupp, lineitem "
        "where s_suppkey = l_suppkey and ps_suppkey = l_suppkey and ps_partkey = l_partkey "
        "group by s_suppkey "
        "order by count"
    )
    '''
    result = np.array(db_connect(sql))
    sorted_result = result[(-result[:, 1]).argsort()]
    u, t = np.hsplit(sorted_result, 2)
    u = u.flatten()
    t = t.flatten()
    t = t.astype(float)
    return u, t

In [4]:
def count(t, tau):
    f = [0] * (2 * tau + 1)
    j = 0
    while j <= 2 * tau:
        f[j] = sum(t[:-j])
        j+=1
    return f

In [5]:
def shifted_inverse(f_k, PARAM):
    # Initialize s with a default value of -tau - 1 for each element
    s = [-PARAM["tau"] - 1] * PARAM["D"]

    for r in range(PARAM["D"]):
        if r == f_k[PARAM["tau"]]:
            s[r] = 0
        else:
            # Use binary search since f_k is in descending order
            j = binary_search(f_k, r)
            if j != -1:
                if 1 <= j <= PARAM["tau"]:
                    s[r] = -PARAM["tau"] + j - 1
                elif PARAM["tau"] < j <= 2 * PARAM["tau"]:
                    s[r] = PARAM["tau"] - j

    # Calculate probabilities proportional to exp(epsilon / 2 * s(V, r))
    p = np.array([np.exp(PARAM["epsilon"] / 2 * s[r]) for r in range(PARAM["D"])])
    p /= p.sum()

    # Sample r from [D] with probability proportional to exp(epsilon / 2 * s(V, r)), denoted by r_tilde
    r_tilde = np.random.choice(PARAM["D"], p=p)

    # Return M(V) = r_tilde
    return r_tilde

In [6]:
def binary_search(f_values, target):
    left, right = 1, 2 * tau
    while left <= right:
        mid = (left + right) // 2
        if f_values[mid] < target <= f_values[mid - 1]:
            return mid
        elif target > f_values[mid - 1]:
            right = mid - 1
        else:
            left = mid + 1
    return -1

In [7]:
# count
def get_error(t, r_tilde):
    r = np.sum(t)
    relative_error = (r - r_tilde) / r
    print("The relative error: {}".format(relative_error))

In [8]:
PARAM = {
    "beta": 1 / 3,
    "epsilon": 4,
    "D": pow(10, 5)
}
tau = math.ceil(2 * PARAM["epsilon"] * math.log((PARAM["D"] + 1) / PARAM["beta"]))
PARAM["tau"] = tau
user, query_tuple = get_query_result()

f = count(t=query_tuple, tau=PARAM["tau"])
r_tilde = shifted_inverse(f_k=f, PARAM=PARAM)
print("The result r_tidle: {}".format(r_tilde))

# evaluate relative error
get_error(t=query_tuple, r_tilde=r_tilde)

The result r_tidle: 74359
The relative error: 0.00645359557467732
